In [2]:
import pyupbit
import time
from collections import deque
import datetime
import matplotlib.pyplot as plt
import datetime
tickers = []
ma5 = []
ma20 = []
ma60 = []

# 원화로 거래 가능한 코인 리스트
tickers = pyupbit.get_tickers(fiat = 'KRW')
# ['KRW-BTC', 'KRW-ETH', 'KRW-NEO', 'KRW-MTL', 'KRW-LTC'] ... 등등

# 로그인 정보 저장
upbit = pyupbit.Upbit('7dtWPaskQ2C2SGSa240M76QylZZEJvx5IjLSP1Vx', 'J57lzm4ppRwOtlhFo3Och3DtxvEI8PvbKxly9xj7')

# 잔고 조회 krw
def get_balance_krw():    
    balance = upbit.get_balance("KRW")
    return balance

# 잔고 조회 coin
def get_balance_wallet(ticker):
    balances = upbit.get_balances()
    b = True
    while (b):
        for i in range(len(balances)):
            a = ticker[4:7]
            if balances[i]['currency'] == a:
                balance = balances[i]['balance']
                avg_buy_price = balances[i]['avg_buy_price']
                c = (float(avg_buy_price), float(balance))
                b = False
            else:
                c = (int(0), int(0))
                b = False
    return c

def get_ticker_ma(ticker):

    '''get_ohlcv 함수는 고가/시가/저가/종가/거래량을 DataFrame으로 반환합니다'''
    df = pyupbit.get_ohlcv(ticker, interval='minute5') # 5분봉 데이터 프레임 생성
    
    # 코인 종가를 담을 deque 변수
    ma5 = deque(maxlen = 5)
    ma20 = deque(maxlen = 20)
    ma60 = deque(maxlen = 60)

    ma5.extend(df['close'])    # ma5 변수에 종가 넣기
    ma20.extend(df['close'])    # ma20 변수에 종가 넣기
    ma60.extend(df['close'])   # ma60 변수에 종가 넣기

    df['date'] = df.index

    ma5 = df['close'].rolling(window=5).mean()
    df.insert(len(df.columns), "MA5", ma5)

    ma20 = df['close'].rolling(window=20).mean()
    df.insert(len(df.columns), "MA20", ma20)

    ma60 = df['close'].rolling(window=60).mean()
    df.insert(len(df.columns), "MA60", ma60)
    
    now_price = pyupbit.get_current_price(ticker)       # 코인의 현재가
    open_price = df['open'][-1]                 # 당일 시가 구하기
    coin_check = get_balance_wallet(ticker) # 코인 보유 하고 있는지 체크
    avg_price = coin_check[0]   # 매수 평균가
    balance = coin_check[1]         # 코인 보유 개수

    # 매수 평균가가 int 이면 매수 조건 체크 float이면 매도 조건 체크
    if type(avg_price) == int:

        if df['MA5'][-1] > df['MA20'][-1]:
            # 6천원치 매수
            volume = 8000
            buy_order(ticker, volume)
        else:
            print('시세 감시 중')
            pass
    else:
        # 현재 보유 코인 수익률 계산
        buy_profit = ((now_price - avg_price)/avg_price) * 100
        profit = round(buy_profit, 2)

        if df['MA5'][-1] < df['MA20'][-1] or avg_price > now_price:
            sell_order(ticker, balance)
        else:
            print(f"코인명: {ticker}, 수익률: {profit}%")
            print("내가산 가격 = {}, 시장가격 = {}".format(avg_price, now_price))

# record(AAPL=data.current(context.sym, "price"), ma5=ma5, ma20=ma20, buy=buy, sell=sell)

# algo = TradingAlgorithm(initialize=initialize, handle_data=handle_data)
# result = algo.run(data)

# 매수 주문
def buy_order(ticker, volume):
    try:
        while True:
            buy_result = upbit.buy_market_order(ticker, volume)
            if buy_result == None or 'error' in buy_result:
                print("매수 재 주문")
                time.sleep(1)
            else:
                print(datetime.datetime.now().strftime('%Y_%m_%d_%H_%M','분', '%S','초'))
                print('매수 체결')
                return buy_result
    except:
        print("매수 주문 이상")

# 매도 주문
def sell_order(ticker, volume):
    try:
        while True:
            sell_result = upbit.sell_market_order(ticker, volume)
            if sell_result == None or 'error' in sell_result:
                print(f"{sell_result}, 매도 재 주문")
                time.sleep(1)
            else:
                print(datetime.datetime.now().strftime('%Y_%m_%d_%H_%M_%S'))
                print('매도 체결')
                return sell_result
    except:
        print("매도 주문 이상")

while True:
    get_ticker_ma(tickers[1])
    time.sleep(30)


시세 감시 중
시세 감시 중
시세 감시 중
시세 감시 중
시세 감시 중
시세 감시 중
시세 감시 중
시세 감시 중
시세 감시 중
시세 감시 중
시세 감시 중
시세 감시 중
시세 감시 중
시세 감시 중
시세 감시 중
시세 감시 중
시세 감시 중
